## <b><font color='darkblue'>Model Context Protocol(MCP) with Google Gemini 2.5 Pro — A Deep Dive (Full Code)</font></b>
([source](https://medium.com/google-cloud/model-context-protocol-mcp-with-google-gemini-llm-a-deep-dive-full-code-ea16e3fac9a3)) <b><font size='3ptx'>A step-by-step guide with code, architecture, and real-world use case</font></b>

As Large Language Models (LLMs) like GPT-4, Claude, Gemini and Llama3 evolve , we need standardized ways to connect them to tools, APIs, and systems .However, these models operate in isolation based on pre-trained data and don’t have built-in access to real-time data, databases , external APIs, or local files.

<b>In this article , we’ll discuss more on Model Context Protocol (MCP) concept before diving into implementation :</b>

### <b><font color='darkgreen'>1. What is Model Context Protocol (MCP) ?</font></b>
<font size='3ptx'><b>The [**Model Context Protocol (MCP)**](https://modelcontextprotocol.io/introduction) is a standardized, open protocol developed by Anthropic that enables AI models to seamlessly interact with external data sources and tools</b>, acting as a universal connector for AI integrations.</font>
> Think of MCP as a “USB-C for AI integrations,” providing a universal way for AI models to connect to different devices and data sources

### <b><font color='darkgreen'>How MCP works ?</font></b>
<b><font size='3ptx'>MCP follows a client-server architecture, where</font></b>:
* <b>Clients</b> (like AI applications or LLMs) connect to
* <b>Servers</b> (MCP tool providers) expose tools, APIs, or data sources to clients.

This enables dynamic and structured interactions between LLM models and the external API’s.

### <b><font color='darkgreen'>Benefits of MCP.</font></b>
MCP transforms the API into a model-friendly tool, complete with auto-discovery, a predictable schema, and structured interaction.

1. <b><font size='3ptx'>Standardized Integration</font></b>: Connect LLMs to any external system with less custom work.
2. <b><font size='3ptx'>Flexibility</font></b>: LLMs can use multiple tools and services — on demand.
3. <b><font size='3ptx'>Security</font></b>: Supports secure API interaction without hardcoding credentials.
4. <b><font size='3ptx'>Simplified Development</font></b>: Build and expose custom MCP servers easily.
5. <b><font size='3ptx'>Easier Maintenance</font></b>: No more repetitive integration logic.

### <b><font color='darkgreen'>Examples of MCP Servers</font></b>
* <b><font size='3ptx'>File System</font></b>: Accessing local files and directories.
* <b><font size='3ptx'>Web Search</font></b>: Run real-time web searches.
* <b><font size='3ptx'>Databases</font></b>: Query SQL or NoSQL databases
* <b><font size='3ptx'>CRMs</font></b>: Connecting to CRM systems like Salesforce.
* <b><font size='3ptx'>Version Control</font></b>: Accessing version control systems like Git

### <b><font color='darkgreen'>When to use Model Context Protocol (MCP) ?</font></b>
MCP to be used when:
* We’re building <b><font size='3ptx'>agentic systems</font></b>
* We want tools to <b><font size='3ptx'>be modular, reusable, discoverable</font></b>
* We want use multiple external sources -
* We want scaling to <b><font size='3ptx'>multiple tools or toolchains</font></b>

### <b><font color='darkgreen'>Architecture</font></b>
Below project integrates multiple components to enable natural language flight search using Gemini + MCP:
![MCP Architecture](images/1.png)

#### <b>Component Interactions</b>
1. <b><font size='3ptx'>User to Client</font></b>
    - User provides natural language query (<font color='brown'>e.g., “Find flights from Atlanta to Las Vegas tomorrow”</font>)
    - Client script (`client.py`) processes the input
2. <b><font size='3ptx'>Client to MCP Server</font></b>
    - Client starts the MCP server process (`mcp-flight-search`)
    - Establishes stdio communication channel.
    - Retrieves available tools and their descriptions
3. <b><font size='3ptx'>Client to Gemini API</font></b>
    - Sends the user’s query
    - Provides tool descriptions for function calling
    - Receives structured function call with extracted parameters
4. <b><font size='3ptx'>Client to MCP Tool</font></b>
    - Takes function call parameters from Gemini
    - Calls appropriate MCP tool with parameters
    - Handles response processing
5. <b><font size='3ptx'>MCP Server to SerpAPI</font></b>
    - MCP server makes requests to SerpAPI
    - Queries Google Flights data
    - Processes and formats flight information

## <b><font color='darkblue'>Implementation</font></b>
([Github link of codes](https://github.com/arjunprabhulal/mcp-gemini-search)) <b><font size='3ptx'>Let us dive into building this pipeline with Gemini AI by breaking down into key implementation steps</font></b>

### <b><font color='darkgreen'>Pre-Requisites</font></b>
1. Python 3.8+ installed
2. [**Google Gemini**](https://makersuite.google.com/app) Generative AI access via [API key](https://aistudio.google.com/apikey)
3. A valid [**SerpAPI key**](https://serpapi.com/) (used to fetch live flight data)

### <b><font color='darkgreen'>Step 1 : Setup virtual environment</font></b>

#### <b>Install the dependancies</b>
* [**google-genai**](https://ai.google.dev/gemini-api/docs/libraries): The official Python library for interacting with Google's Generative AI models (like Gemini).
* [**mcp**](https://github.com/modelcontextprotocol/python-sdk): A Python SDK for interacting with an MCP (Model Context Protocol) server. This SDK likely provides functionalities to communicate with external tools or services.

```shell
#Setup virtual env
$ python -n venv venv 

#Activate venv
$ source venv/bin/activate

#Install dependancies
$ pip install google-genai mcp
```

In [4]:
!pip freeze | grep -P "(google-genai|mcp)"

fastmcp==2.11.2
google-genai==1.26.0
langchain-google-genai==2.1.6
langchain-mcp-adapters==0.0.7
mcp==1.12.0
mcp-flight-search==0.2.1


#### <b>Set Environment variables</b>

```shell
export GEMINI_API_KEY="your-google-api-key"
export SERP_API_KEY="your-serpapi-key"
```

### <b><font color='darkgreen'>Step 2: Install the MCP Server — mcp-flight-search</font></b>
<b><font size='3ptx'>To enable Gemini to interact with real-world APIs, we’ll use an MCP-compliant server.</font></b>

For this article, we’ll use [**mcp-flight-search**](https://github.com/arjunprabhulal/mcp-flight-search) — a lightweight MCP Server built using [**FastMCP**](https://github.com/jlowin/fastmcp) which exposes a tool that searches real-time flight data using the SerpAPI. Install MCP server package where I published to PyPi:
```shell
# Install from PyPI
$ pip install mcp-flight-search
```

Let us verify if MCP server package is installed successfully:

In [5]:
!pip show 'mcp-flight-search'

Name: mcp-flight-search
Version: 0.2.1
Summary: Flight search service implementing  Model Context Protocol (MCP) tools
Home-page: https://github.com/arjunprabhulal/mcp-flight-search
Author: 
Author-email: Arjun Prabhulal <code.aicloudlab@gmail.com>
License: 
Location: /usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.12/site-packages
Requires: fastmcp, google-search-results, pydantic, python-dotenv, rich
Required-by: 


### <b><font color='darkgreen'>Step 3 : Understanding MCP Tool Packages</font></b>
<b><font size='3ptx'>Import library which initializes both Gemini and MCP SDKs and prepares for async execution...</font></b>

In [6]:
from google import genai

Above imports the <b>genai</b> module from the <b>google-generativeai</b> library. It <b><font size='3ptx'>provides access to Google’s powerful LLMs, such as Gemini 1.5 and 2.0,2.5 model and includes client methods to interact with models using natural language</font></b>.

In [7]:
from google.genai import types

Above module gives access to the <b>type definitions and configuration structures</b> used by the Gemini API. For example:
- <b>Tool</b>: Defines tools (functions) that the model can call.
- <b>GenerateContentConfig</b>: Allows us to configure how the model responds (e.g., temperature, tool support, etc.).

In [8]:
from mcp import ClientSession, StdioServerParameters

Above classes come from the [**mcp-sdk-python**](https://pypi.org/project/mcp-sdk-python/) library and are essential for interacting with <b>MCP servers</b>:
- <b><font size='3ptx'>ClientSession</font></b>: Manages the communication session between our client/app and the MCP server.
- <b><font size='3ptx'>StdioServerParameters</font></b>: stdio allows the server to be language-neutral and easily embedded in different environments.

In [9]:
from mcp.client.stdio import stdio_client

This imports the <font size='3ptx'><b>`stdio_client`, an asynchronous context manager used to establish a connection with an MCP server over standard I/O</b></font>. It ensures that the server is correctly launched, and the client is ready to send/receive structured requests.

Above 4 key imports together form the backbone of how we <b><font size='3ptx'>bridge Gemini’s LLM interaction with real-world APIs exposed via MCP tools</font></b>.

### <b><font color='darkgreen'>Step 4: Initialize Gemini Client</font></b>
<font size='3ptx'><b>`genai.Client()`</b> is the primary interface used to interact with Google’s generative models (<font color='brown'>e.g., Gemini 2.5 Pro, Gemini 2 Flash</font>)</font>

In [10]:
import os

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Once GenAI Client is initialized, this client object can:
* Send prompts to Gemini models
* Pass tool definitions (function calling)
* Receive structured responses and function call objects

### <b><font color='darkgreen'>Step 5 : Configure MCP Tool Server</font></b>
<font size='3ptx'>Below block sets up the parameters required to <b>launch and communicate with the MCP server that exposes tools</b> (<font color='brown'>in our case, a flight search function</font>).</font>

In [11]:
server_params = StdioServerParameters(
    command="mcp-flight-search",
    args=["--connection_type", "stdio"],
    env={"SERP_API_KEY": os.getenv("SERP_API_KEY")},
)

- <b><font size='3ptx'>mcp-flight-search</font></b> — This is the CLI entry point to run local MCP server,or could be a Python module in our case that implements the MCP protocol.

In [12]:
!mcp-flight-search -h

usage: mcp-flight-search [-h] [--connection_type {http,stdio}] [--port PORT]

Model Context Protocol Flight Search Service

options:
  -h, --help            show this help message and exit
  --connection_type {http,stdio}
                        Connection type (http or stdio)
  --port PORT           Port to run the server on (default: 3001)


* <b><font size='3ptx'>--connection_type stdio</font></b>: This tells the server to <b>use standard input/output (stdio) as its communication channel</b>. Stdio is simple, language-agnostic, and great for running tool servers locally or in subprocesses.

* <b><font size='3ptx'>SERP_API_KEY</font></b> — This passes an environment variable (`SERP_API_KEY`) to the subprocess running the tool. In our case, the tool needs it to authenticate with SerpAPI, which fetches real-time flight data.


Once `server_params` is defined, we can use it to spin up the server using the `stdio_client` async context manager.
```python
>>> import os
>>> from google import genai
>>> from google.genai import types
>>> from mcp import ClientSession, StdioServerParameters
>>> from mcp.client.stdio import stdio_client
>>>
>>> client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
>>>
>>> server_params = StdioServerParameters(
...     command="mcp-flight-search",
...     args=["--connection_type", "stdio"],
...     env={"SERP_API_KEY": os.getenv("SERP_API_KEY")},
... )
>>> server_params
StdioServerParameters(command='mcp-flight-search', args=['--connection_type', 'stdio'], env={'SERP_API_KEY':'XXXXXXXXX'}, cwd=None, encoding='utf-8', encoding_error_handler='strict')
```

The <b>Gemini client</b> handles language understanding, prompt generation, and function calling; The <b>MCP tool server</b> (flight search) listens for tool calls and executes them in real time via SerpAPI.

### <b><font color='darkgreen'>Step 6 : Connecting to MCP Server and listing tools</font></b>
<b><font size='3ptx'>Below block of code does three important steps</font></b>
1. Starts connection with the MCP server ,
2. Initializes a session for structured tool communication and
3. Dynamically discovers and formats available tools for Gemini.

```python
async def run():
    # Remove debug prints
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            prompt = f"Find Flights from Atlanta to Las Vegas 2025-05-05"
            await session.initialize()
            # Remove debug prints

            mcp_tools = await session.list_tools()
            # Remove debug prints
            tools = [
                types.Tool(
                    function_declarations=[
                        {
                            "name": tool.name,
                            "description": tool.description,
                            "parameters": {
                                k: v
                                for k, v in tool.inputSchema.items()
                                if k not in ["additionalProperties", "$schema"]
                            },
                        }
                    ]
                )
                for tool in mcp_tools.tools
            ]
            # Remove debug prints

            response = client.models.generate_content(
                model="gemini-2.5-pro-exp-03-25",
                contents=prompt,
                config=types.GenerateContentConfig(
                    temperature=0,
                    tools=tools,
                ),
            )
```

Let us breakdown line-by-line to understand how the MCP Client-Server communication is happening under the hood along with Gemini LLM.

**`stdio_client`** is an asynchronous context manager that handles:
* Launching the MCP server as a subprocess.
* Managing the input/output streams for message exchange

**`read` and `write`** objects are asynchronous streams:
* **`read`**: reads responses or tool registration from the server
* **`write`**: sends requests or tool invocations to the server

In [13]:
prompt = f"Find Flights from Atlanta to Las Vegas 2025-05-05"

Above prompt is <b><font size='3ptx'>natural language query</font></b> we’ll send to the Gemini model. which Gemini will later turn into a structured tool call.

Line `await session.initialize()` is the one which <b><font size='3ptx'>triggers the initial MCP handshake between our client and the server</font></b>. server registers its available tools. From this step, it achieves:
* Server registers its available tools (in our case: a **flight search tool**).
* Session is now ready to list, call, and execute tools.

For line `mcp_tools = await session.list_tools()`, it requests the list of all tools (functions) exposed by the server. Each tool in **`mcp_tools.tools`** contains:
* A name
* A description
* An input schema (<font color='brown'>i.e., what parameters it accepts, in JSON Schema format</font>)

**`mcp_tools.tools`** makes the MCP server self-describing, so that LLM can automatically understand how to call each tool.

Regarding below codesnippet:
```python
tools = [
    types.Tool(
        function_declarations=[
            {
                "name": tool.name,
                "description": tool.description,
                "parameters": {
                    k: v
                    for k, v in tool.inputSchema.items()
                    if k not in ["additionalProperties", "$schema"]
                },
            }
        ]
    )
    for tool in mcp_tools.tools
]
```

It converts the MCP tool definitions into [**Gemini’s function_declarations format**](https://ai.google.dev/gemini-api/docs/function-calling?example=weather#function_declarations). Now that our MCP server is running and session is initialized to discover tools from MCP Server for Gemini to use

### <b><font color='darkgreen'>Step 7 : Gemini — Interprets Prompt and suggest a Function Call</font></b>
<font size='3ptx'>Finally the <b>user’s prompt is sent to the Gemini model, along with a list of available tools discovered from the MCP server</b></font> by below code snippet:
```python
response = client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=0,
        tools=tools,
    ),
)
```

If Gemini recognizes the prompt as matching a `function’s schema`, it returns a `function_call` object that includes the tool name and the auto-filled parameters. If Gemini determines that the prompt aligns with a function (based on name, description, or parameters), it returns a structured `function_call` object like:
```json
{
  "function_call": {
    "name": "search_flights",
    "args": {
      "source": "ATL",
      "destination": "LAS",
      "date": "2025-05-05"
    }
  }
}
```

Gemini LLM transitions from a passive text model to an active decision-maker that:
* Interprets natural input
* Selects an appropriate tool
* Fills in the function’s arguments automatically
* We didn’t write any parsing logic.
* Gemini LLM model filled in all the fields by interpreting the user’s natural language.
* function call is structured and ready for execution.

Below code snippet show how to call the tool:
```python
result = await session.call_tool(
    function_call.name, arguments=dict(function_call.args)
)
```

### <b><font color='darkgreen'>Final Demo: Gemini 2.5 Pro with MCP</font></b>
Below debug logs show exactly how Gemini , Model Context Protocol (MCP) work together to interpret user intent, match a tool, and return real-time data.
![flow](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*D0OHnjompVnvPNGBUdbfmQ.gif)

Or you can try local script `client.py`:
```shell
$ ./client.py 
...
--- Formatted Result ---
{
  "airline": "Delta",
  "price": "349",
  "duration": "266 min",
  "stops": "Nonstop",
  "departure": "Hartsfield-Jackson Atlanta International Airport (ATL) at 2026-05-05 09:50",
  "arrival": "Harry Reid International Airport (LAS) at 2026-05-05 11:16",
  "travel_class": "Economy",
  "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/DL.png"
}
```

## <b><font color='darkblue'>Best Practices for Using Model Context Protocol (MCP) with Gemini LLM</font></b>

### <b><font color='darkgreen'>1. Tool Design</font></b>
* <b><font size='3ptx'>Clear Tool Names</font></b>: Use short, meaningful names (<font color='brown'>e.g., `search_flights`, `get_weather`</font>).
* <b><font size='3ptx'>Describe Each Tool Well</font></b>: Provide simple, clear descriptions — the model uses this to decide when and how to call the tool.
* <b><font size='3ptx'>Use Strong Typing</font></b>: Define input parameters explicitly (<font color='brown'>e.g., string, enum, number</font>) to help the model fill them accurately.

### <b><font color='darkgreen'>2. Model Interaction</font></b>
* <b><font size='3ptx'>Fewer Tools means Better Accuracy</font></b>: Avoid overloading the model — stick to relevant tools only.
* <b><font size='3ptx'>Dynamic Tool Loading</font></b>: Load tools based on the user’s query or conversation context.
* <b><font size='3ptx'>Prompt the Model Clearly</font></b>: Set the model’s role and explain how and when to use the tools.

### <b><font color='darkgreen'>3. Server Setup</font></b>
* <b><font size='3ptx'>Use stdio for Simplicity</font></b>: Start MCP servers using — connection_type stdio for easy local development.
* <b><font size='3ptx'>Pass Environment Variables Safely</font></b>: Use `env` to send keys like `SERP_API_KEY` securely to tool servers.

### <b><font color='darkgreen'>4. Request Handling</font></b>
* <b><font size='3ptx'>Initialize Session First</font></b>: Always run `session.initialize()` before listing or calling tools.
* <b><font size='3ptx'>List Tools Dynamically</font></b>: Use `session.list_tools()` to keep client flexible and tool-agnostic.

### <b><font color='darkgreen'>5. Error Handling & Security</font></b>
* <b><font size='3ptx'>Return Helpful Errors</font></b>: Make sure tool server responds with meaningful messages when something fails.
* <b><font size='3ptx'>Secure APIs</font></b>: Never expose secrets like API keys in logs or error messages.

### <b><font color='darkgreen'>Limitations</font></b>
As of this writing( March 2025), there are a few limitations when using <b><font size='3ptx'>Model Context Protocol (MCP)</font></b> and function calling with Gemini LLM:
1. Partial OpenAPI Support
2. Supported parameter types in Python are limited.
3. Automatic function calling is a Python SDK feature only.